Alternative Skolemization Ideas
==================

UNDER CONSTRUCTION

This is a temporary notebook for exploring issues related to a skolemize() method for KnownTruth objects, in particular for known truths involving universal ($\forall$) and existential ($\exists$) quantifiers, with skolemization eventually being part of a more general instantiation or specialization method.

The notebook was generated initially as a copy of the Forall tutorial notebook, and thus while under construction many of the cells and other elements of the original notebook remain.

We import some necessary information then consider an example of a `Forall` object like $\forall_{x \,\in\, S \,|\, Q(x), R(x)} P(x)$:

In [ ]:
from proveit import Function, ExprList, Lambda, Literal
from proveit.logic import Forall, Exists, InSet, Equals
from proveit.logic.equality._axioms_ import substitution
from proveit.number import Less, Add, Frac, zero
from proveit._common_ import a, b, c, f, x, y, z, fx, P, Px, Pxy, Q, Qx, R, Rx, Ry, S, t, T
%begin skolemization_explorations

In [1]:
# intended to operate on expression, not full known truth
# an initial effort to allow skolemization of nested Exists expressions
# and allow skipping over Exists in the nesting
# see skolemize3 and skolemizeHelper for further inroads to the problem
# using a recursive approach
def skolemize2(inputExpr, skMap, nonSkVars=set()):
    from proveit.logic import Exists
    setOfResults = [inputExpr]
    print("setOfResults = ", setOfResults)
    print("len(setOfResults) = ", len(setOfResults))
    
    while len(skMap)>0:
        tempResults = []
        # currentSkMapping = {}
        for theExpr in setOfResults:
    
            # check if outer expression is an Exists operation
            if isinstance(theExpr, Exists):
                print("1 Input was an Exists operation.")
                print("    1.1 the input was theExpr = ", theExpr)
                tempInstanceVar = theExpr.instanceVar
                print("    1.2 with tempInstanceVar = ", tempInstanceVar)
                
                # check if skolemization appropriate
                # i.e. if instance variable appears in skMap as a key
                if tempInstanceVar in skMap:
                    print("    1.3 Found the instanceVar as a key in the map!")
                    # perform skolemization
                    tempInstanceExpr = theExpr.instanceExpr
                    print("        theExpr.tempInstanceExpr = ", tempInstanceExpr)
                    tempInstanceExprSubbed = tempInstanceExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]})
                    print("        tempInstanceExprSubbed = ", tempInstanceExprSubbed)
                    tempResults.append(tempInstanceExprSubbed)
                    print("        tempResults = ", tempResults)
                    tempConditions = theExpr.conditions
                    print("        tempConditions = ", tempConditions)
                    tempConditionsSubbed = []
                    tempConditionsCompletelySubbed = []
                    tempConditionsIncompletelySubbed = []
                    for oldExpr in tempConditions:
                        # interestingly here: the subbed conditions will not necessarily reduce to 
                        # completely skolemized conditions, and thus must be checked
                        # if completely skolemized, then continue as originally planned, appending
                        # them to the tempResults;
                        # if not completely skolemized, hold them separately in another list
                        # which will eventually need to be used in the “else” portion below
                        # can eventually delete the next line
                        newExpr = oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]})
                        # check if vars in newExpr include any vars in nonSkVars
                        print("        vars appearing in newExpr = ", newExpr.freeVars())
                        print("        vars appearing in nonSkVars = ", nonSkVars)
                        print("        vars in intersection: ", nonSkVars.intersection(newExpr.freeVars()))
                        tempIntersection = nonSkVars.intersection(newExpr.freeVars())
                        if len(tempIntersection)==0:
                            tempConditionsCompletelySubbed.append(newExpr)
                        else:
                            tempConditionsIncompletelySubbed.append(newExpr)
                            
                        tempConditionsSubbed.append(oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]}))
                        # the following used to produce flat list
                        tempResults.append(oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]}))
                        
                    print("        tempConditionsCompletelySubbed = ", tempConditionsCompletelySubbed)
                    print("        tempConditionsIncompletelySubbed = ", tempConditionsIncompletelySubbed)
                    # we make a list of lists, consisting of the
                    # instanceExpressionSubbed, tempConditionsCompletelySubbed, tempConditionsIncompletelySubbed
                    # tempResults.append(tempConditionsCompletelySubbed)
                    # tempResults.append(tempConditionsIncompletelySubbed)
                    print("        tempResults = ", tempResults)
                    # remove the Skolem constant from the skMap
                    del skMap[tempInstanceVar]
                    print("        skMap now = ", skMap)
                    # setOfResults = tempResults
                    print("        tempResults now = ", tempResults)
                    
                else:
                    print("    1.4 Did NOT find the instanceVar as a key in the map!")
                    nonSkVars.add(tempInstanceVar) # adding to a SET
                    # try constructing the input expression around an attempted
                    # skolemization on an inner expression (if it exists)
                    tempInstanceExpr = theExpr.instanceExpr
                    print("        tempInstanceExpr = ", tempInstanceExpr)
                    # for testing; delete the following line later
                    # print("    1.4 skolomize2() = ", skolemize2(tempInstanceExpr, skMap))
                    tempSkolemizeResult = skolemize2(tempInstanceExpr, skMap, nonSkVars)
                    print("    1.4 skolomize2() = ", tempSkolemizeResult)
                    A = tempSkolemizeResult[0]  # the 1st item
                    B = tempSkolemizeResult[1:] # all other items
                    print("        A = ", A)
                    print("        B = ", B)
                    print("        just after A, B, tempResults = ", tempResults)
                    tempConstructed = Exists(theExpr.instanceVar,
                                             A,
                                             conditions=theExpr.conditions)
                    print("        tempConstructed = ", tempConstructed)
                    tempResults.append(tempConstructed)
                    print("        tempResults = ", tempResults)
                    tempResults.extend(B)
                    print("        tempResults = ", tempResults)
                    

            else:
                print("2 Input was NOT an Exists operation.")
                print("    2.1 input was theExpr = ", theExpr)
                tempResults.append(theExpr) # perhaps we need to acknowledge a possible substitution?
                print("    2.2 tempResults = ", tempResults)
        
        # end for loop

        setOfResults = tempResults
    
    # end while loop
    
    print("Returning from skolomize2 call, setOfResults = ", setOfResults)
    return setOfResults

In [2]:
def skolemize3(inputExpr, skMap):
    '''
    inputExpr should be an Exists or nested Exists expressions;
    skMap is a dictionary of the form {x:La, y:Lb, ...} specifying
    the desired mapping(s) from Exists instance variable(s) to
    constant Literal(s).
    This is part of an extended version of skolemize meant to allow
    processing of nested Exists expressions and the possible skipping
    over of items in the nesting. This is a recursive approach that
    uses the skolemizeHelper() function.
    '''
    
    # from proveit.logic import Exists
    setOfResults = [inputExpr]

    while len(skMap)>0:
        tempResults = skolemizeHelper(setOfResults, skMap)
        setOfResults = [tempResults[0], *tempResults[1]]
        skMap = tempResults[3]
    # end while loop

    return setOfResults

In [3]:
def skolemizeHelper(setOfResults, skMap, nonSkVars=set()):
    '''
    used by skolemize3() function to perform a recursive skolemization
    on a possibly nested sequence of Exists functions where we might also
    want to skip over some of the nested Exists and skolemize just some
    subset
    '''
    
    from proveit.logic import Exists
    
    tempResults = []
    tempConditionsCompletelySubbed = []
    tempConditionsIncompletelySubbed = []
    
    for theExpr in setOfResults:

        # check if outer expression is an Exists operation
        if isinstance(theExpr, Exists):
            tempInstanceVar = theExpr.instanceVar

            # check if skolemization appropriate
            # i.e. if instance variable appears in skMap as a key
            if tempInstanceVar in skMap:
                # perform skolemization
                tempInstanceExpr = theExpr.instanceExpr
                tempInstanceExprSubbed = tempInstanceExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]})
                tempResults.append(tempInstanceExprSubbed)
                tempConditions = theExpr.conditions
                tempConditionsSubbed = []
                for oldExpr in tempConditions:
                    # Notice that any resulting subbed conditions will not necessarily reduce to 
                    # completely Skolemized conditions, and thus must be checked.
                    # If completely skolemized, append them to the completely subbed conditions;
                    # if not completely skolemized, hold them in the separate incompletely
                    # subbed conditions list which will eventually need to be used in the “else”
                    # portion below
                    newExpr = oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]})
                    # check if vars in newExpr include any vars in nonSkVars
                    tempIntersection = nonSkVars.intersection(newExpr.freeVars())
                    if len(tempIntersection)==0:
                        tempConditionsCompletelySubbed.append(newExpr)
                    else:
                        tempConditionsIncompletelySubbed.append(newExpr)

                    # probably don't need following line anymore
                    # tempConditionsSubbed.append(oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]}))

                # remove the Skolem constant from the skMap
                del skMap[tempInstanceVar]

            else:
                nonSkVars.add(tempInstanceVar) # adding to a SET
                # try constructing the input expression around an attempted
                # skolemization on an inner expression (if it exists)
                tempInstanceExpr = theExpr.instanceExpr
                tempSkolemizeResult = skolemizeHelper([tempInstanceExpr], skMap, nonSkVars)
                A = tempSkolemizeResult[0]  # the 1st item
                B = tempSkolemizeResult[1]  # completely subbed conditions
                C = tempSkolemizeResult[2]  # incompletely subbed conditions
                if len(C) != 0:
                    theExprConditions = ExprList(*theExpr.conditions, *C)
                else:
                    theExprConditions = theExpr.conditions
                tempConstructed = Exists(theExpr.instanceVar,
                                         A,
                                         conditions=theExprConditions)
                tempResults.append(tempConstructed)
                # tempResults.append(B)
                tempConditionsCompletelySubbed.append(B)


        else:
            # tempResults.append(theExpr) # perhaps we need to acknowledge a possible substitution?
            # instead, pop them back into the tempConditionsCompletelySubbed set
            tempConditionsCompletelySubbed.append(theExpr)

    # end for loop
    
    # we compile the results, consisting of the
    # instanceExpressionSubbed, tempConditionsCompletelySubbed,
    # tempConditionsIncompletelySubbed, and the (possibly modified) skMap
    tempResults.append(tempConditionsCompletelySubbed)
    tempResults.append(tempConditionsIncompletelySubbed)
    tempResults.append(skMap)

    return tempResults

In [46]:
basicThereExistsExpr00 = Exists(x, Px)

basicThereExistsExpr00: exists_{x} P(x)

In [47]:
basicThereExistsExpr00KT = basicThereExistsExpr00.prove(assumptions={basicThereExistsExpr00})

basicThereExistsExpr00KT: {exists_{x} P(x)} |- exists_{x} P(x)

In [48]:
basicThereExistsExpr01 = Exists(x, Px, conditions=[Qx], domain=S)

basicThereExistsExpr01: exists_{x in S | Q(x)} P(x)

In [49]:
basicThereExistsExpr01KT = basicThereExistsExpr01.prove(assumptions={basicThereExistsExpr01})

basicThereExistsExpr01KT: {exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [50]:
basicThereExistsExpr02 = Exists([x, y], Pxy, conditions=[Qx, Ry], domain=S)

basicThereExistsExpr02: exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [51]:
basicThereExistsExpr02KT = basicThereExistsExpr02.prove(assumptions = {basicThereExistsExpr02} )

basicThereExistsExpr02KT: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [52]:
Txy = Function(T, {x, y})
basicThereExistsExpr03 = Exists([x, y], Pxy, conditions=[Txy, Qx, Ry], domain=S)

basicThereExistsExpr03: exists_{x, y in S | T(x , y) , Q(x) , R(y)} P(x , y)

In [25]:
basicThereExistsExpr03.instanceExpr.conditions

(y in S , T(x , y) , Q(x) , R(y))

In [53]:
(basicThereExistsExpr03.instanceExpr.allConditions()[1]).substituted({z:a})

T(x , y)

In [54]:
Tz = Function(T, z)
Pxyz = Function(P, [x, y, z])
basicThereExistsExpr04 = Exists([x, y, z], Pxyz, conditions=[Qx, Ry, Tz], domain=S)

basicThereExistsExpr04: exists_{x, y, z in S | Q(x) , R(y) , T(z)} P(x , y , z)

In [55]:
Tz = Function(T, z)
Pxyz = Function(P, [x, y, z])
Qxyz = Function(Q, [x, y, z])
basicThereExistsExpr05 = Exists([x, y, z], Pxyz, conditions=[Qxyz], domain=S)

basicThereExistsExpr05: exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [56]:
basicThereExistsExpr05KT = basicThereExistsExpr05.prove(assumptions = {basicThereExistsExpr05} )

basicThereExistsExpr05KT: {exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [57]:
Pxyzt = Function(P, [x, y, z, t])
Qxyzt = Function(Q, [x, y, z, t])
basicThereExistsExpr06 = Exists([x, y, z, t], Pxyzt, conditions=[Qxyzt], domain=S)

basicThereExistsExpr06: exists_{x, y, z, t in S | Q(x , y , z , t)} P(x , y , z , t)

In [31]:
basicThereExistsExpr00KT

{exists_{x} P(x)} |- exists_{x} P(x)

In [58]:
QLc = Function(Q, Lc)
basicThereExistsExpr00KTSkolemized = basicThereExistsExpr00KT.skolemize({x:Lc}, assumptions={QLc, InSet(Lc, S)})

ENTERING KT.skolemize()
        processedSubMap so far =  {x: c}
ENTERING KT._skolemized_expressions


basicThereExistsExpr00KTSkolemized: {exists_{x} P(x)} |- P(c)

In [34]:
basicThereExistsExpr00KTSkolemized.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x} P(x)} |- P(c)
1	assumption		{exists_{x} P(x)} |- exists_{x} P(x)

In [63]:
basicThereExistsExpr01KT

{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [60]:
Ld = Literal('d')
QLd = Function(Q, Ld)
# A, B, C = (
#     basicThereExistsExpr01KT.skolemize(
#         {x:Ld},
#         assumptions={InSet(Ld, S), QLd}
#     )
# )
basicThereExistsExpr01KTSkolemized = (
    basicThereExistsExpr01KT.skolemize(
        {x:Ld},
        assumptions={InSet(Ld, S), QLd}
    )
)
# skolemizing just one of two possible instance vars
# This is not quite working — somehow subbing into one of the conditions
# but not explicitly then including that condition in the
# subbedConditions list
# basicThereExistsExpr03ThmSkolemized = (
#     basicThereExistsExpr03Thm.skolemize(
#         {y:Ld},
#         assumptions={InSet(Ld, S), RLd}
#     )
# )
A, B, C = basicThereExistsExpr01KTSkolemized

ENTERING KT.skolemize()
        processedSubMap so far =  {x: d}
ENTERING KT._skolemized_expressions


A: {exists_{x in S | Q(x)} P(x)} |- P(d)
B: {exists_{x in S | Q(x)} P(x)} |- d in S
C: {exists_{x in S | Q(x)} P(x)} |- Q(d)

In [37]:
A.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- P(d)
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [38]:
B.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- d in S
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [39]:
C.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- Q(d)
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [113]:
basicThereExistsExpr01KT

{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

Very generally when skolemizing, we need to acknowledge free variables.
For example, in the expression $\forall x \exists y [x+y=0]$, we can skolemize y to a constant $y_0$, but need to acknowledge that that constant depends on x, so we might have something like $x + y_0(x) = 0$.
For skolemizing in Prove-It, however, we let the user decide how to specify the Skolem constant(s) and then constrain the resulting expressions later in the context of possible generalizations. For example, if the user has skolemized as discussed above to get the expression $x + y_0(x) = 0$, we could generalize back to obtain $\forall x \exists y [x+y=0]$. If the original skolemization was less explicit, however, producing something like: $x + y_0 = 0$, attempted generalization would simply fail (and the generalization step would have looked for )

This is how Wayne has described it:
$\ldots$ I think I have a better solution that maximizes flexibility and choice for the user.  Let's allow the user to Skolemize either to a simple constant, or a function with a Skolem constant operator and variable operands (we may want to allow literal operands as well).  Now, the rule is simple.  You are not allowed to perform universal generalization if there are any Skolem constants or Skolem functions except Skolem functions that contain the variable being generalized as an argument.  In other words:

1. If your "known truth" contains one or more Skolem constants, you cannot do universal generalization over any variable.

2. If your "known truth" contains any Skolem functions that do not depend upon x, you cannot universally generalize x.

3. If your "known truth" contains only Skolem functions that do depend upon x (e.g., u0(x)), you can universally generalize x.

This allows us to use Skolem functions when we want and simply use Skolem constants when that is just fine for our purposes.

In [40]:
basicThereExistsExpr02KT

{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [44]:
innerExists = basicThereExistsExpr02KT.instanceExpr

innerExists: exists_{y in S | R(y)} P(x , y)

In [45]:
innerExists.conditions

(y in S , R(y))

In [46]:
innerExists.instanceExpr.substituted({y:a},)

P(x , a)

In [47]:
innerExists.substituted({y:a},)

exists_{y in S | R(y)} P(x , y)

In [48]:
rebuiltExists = Exists((a,), innerExists.instanceExpr.substituted({y:a},), conditions=[innerExists.conditions.substituted({y:a},)])

rebuiltExists: exists_{a | (a in S , R(a))} P(x , a)

In [49]:
innerExists.conditions[0].freeVars()

{S, y}

In [66]:
basicThereExistsExpr04

exists_{x, y, z in S | Q(x) , R(y) , T(z)} P(x , y , z)

In [67]:
tempResult0401 = skolemize3(basicThereExistsExpr04, {y:b, z:c})

NameError: name 'skolemize3' is not defined

In [56]:
tempResult0402 = skolemize3(basicThereExistsExpr04, {x:a, y:b, z:c})

tempResult0402: (P(a , b , c) , c in S , T(c) , b in S , R(b) , a in S , Q(a))

In [57]:
tempResult0403 = skolemize3(basicThereExistsExpr04, {x:a, z:c})

tempResult0403: (exists_{y in S | R(y)} P(a , y , c) , (c in S , T(c)) , a in S , Q(a))

In [58]:
basicThereExistsExpr05

exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [64]:
tempResult0601 = skolemize3(basicThereExistsExpr06, {x:La, y:Lb, z:Lc, t:Ld})

tempResult0601: (P(a , b , c , d) , d in S , Q(a , b , c , d) , c in S , b in S , a in S)

In [65]:
tempResult0602 = skolemize3(basicThereExistsExpr06, {y:Lb, t:Ld})

tempResult0602: (exists_{x in S} [exists_{z in S | Q(x , b , z , d)} P(x , b , z , d)] , ((d in S)) , (b in S))

In [66]:
tempResult0603 = skolemize3(basicThereExistsExpr06, {t:Ld})

tempResult0603: (exists_{x in S} [exists_{y in S} [exists_{z in S | Q(x , y , z , d)} P(x , y , z , d)]] , (((d in S))))

In [67]:
basicThereExistsExpr02KT

{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)